Run install and import code blocks every time 

In [2]:
!pip install netCDF4
!pip install xarray

     |████████████████████████████████| 4.3 MB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 287 kB 54.9 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 720 kB 11.8 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
import xarray as xr
import requests
import netCDF4
import boto3
import matplotlib.pyplot as plt
import datetime as dt
import fsspec
import s3fs

This import statement will run goes_reader.py

In [27]:
import goes_reader.py as goes_reader

FileNotFoundError: [Errno 2] No such file or directory: b'noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590002504_e20170590005277_c20170590005335.nc'

In [4]:
# not using this anymore so just commenting it out for now

# lyr = 2017 # year
# idyjl = 210 # day of year
# d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl)

# fs = s3fs.S3FileSystem(anon=True) # connect to s3 bucket!

# # create strings for the year and julian day
# imon,idym=d.month,d.day
# syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    
# # use glob to list all the files in the directory, for goes-east
# file_location,var = fs.glob('s3://noaa-goes16/ABI-L2-CMIPC/'+syr+'/*/*/*.nc'),'SST'

In [13]:
# test using 2017

file_location = []

fs = s3fs.S3FileSystem(anon=True) # connect to s3 bucket!
    
path = 's3://noaa-goes16/ABI-L2-CMIPC/2017/*/*/*.nc'
    
file_location = fs.glob(path)

file_ob = []

for file in file_location: 
    file_ob.append(fs.open(file)) # open connection to files

In [25]:
file_location[0]

'noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590002504_e20170590005277_c20170590005335.nc'

In [22]:
with fs.open(file) as f:
     print(f.read())

AttributeError: 'S3File' object has no attribute 'rstrip'

In [17]:
file = file_ob[0]
file

<File-like object S3FileSystem, noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590002504_e20170590005277_c20170590005335.nc>

In [20]:
file_location[0]

'noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590002504_e20170590005277_c20170590005335.nc'

In [18]:
type(file)

s3fs.core.S3File

In [ ]:
# loop to grab all data from 2017-2020

yrs = [2017,2018,2019,2020]
file_location = []

for lyr in yrs:
    fs = s3fs.S3FileSystem(anon=True) # connect to s3 bucket!

    # create string for year
    syr = str(lyr).zfill(4)
    
    path = 's3://noaa-goes16/ABI-L2-CMIPC/{}/*/*/*.nc'.format(syr)
    
    # use glob to list all the files in the directory, for goes-east
    file_location.append(fs.glob(path))

In [ ]:
file_ob = []
for lst in file_location: 
    file_ob.append([fs.open(file) for file in lst]) # open connection to files

In [7]:
!pip install meracan-s3netcdf

  Created wheel for meracan-s3netcdf: filename=meracan_s3netcdf-0.0.2-py3-none-any.whl size=12985 sha256=ed5d8ed37bba711365ef86b28ce47533d7ebbff8a655c7b4cb1275092e3f3c47
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a4/7b/76/c81fddac1028bc480dd415dcf1db307838fb727e137dcef369
Successfully built meracan-s3netcdf
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [5]:
import numpy as np
file_ob_array = np.concatenate([np.array(i) for i in file_ob])

In [18]:
import json
data = {
  "name":"input1",
  "cacheLocation":"../s3",
  "bucket":"goes-nc-files",
  "localOnly":True,
  "cacheSize":10.0,
  "ncSize":1.0
}
with open('data.txt', 'w') as inputfile:
    json.dump(data, inputfile)

In [9]:
import json
json.loads('input.json')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [48]:
from s3netcdf import NetCDF2D 

# Create/Open master file
netcdf2d = NetCDF2D(data)

# Writing
netcdf2d["file"] = file_ob_array

Exception: NetCDF2D needs a nca object

In [33]:
?NetCDF2D

In [ ]:
ds = netCDF4.Dataset(fs.cat_file(file_ob[0]), 'r')

In [ ]:
ds = xr.open_mfdataset(file_ob,combine='nested',concat_dim='time') #note file is super messed up formatting